In [ ]:
%pip install "unstructured[all-docs]==0.18.15" "unstructured[local-inference]==0.18.15"

In [ ]:
%pip install "torch==2.4.1" "torchvision==0.19.1" "accelerate==0.28.0" "transformers==4.39.3" "timm==1.0.20" "pillow==11.3.0"

Download Results

This notebook polls Acordis API for completed processing and downloads results to Volumes.



## Imports

In [ ]:
# Import required libraries

from datetime import datetime
import uuid
import json
import pandas as pd
import io
import zipfile
from pathlib import Path

from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql import Window

from pyspark.sql.types import StructType, StructField, StringType, BinaryType, LongType, TimestampType, IntegerType

from unstructured.partition.pdf import partition_pdf
# sys.path.append("../../../src")
# from framework.utils.config import ConfigManager
# from framework.acordis_client import AcordisApiClient

print("✓ Basic imports completed")

## ConfigManager

TODO: Move to framework once packaging issue is resolved

In [ ]:
"""
Configuration management utilities for PDF processing framework.

This module provides utilities for loading, validating, and managing
configuration files used throughout the PDF processing pipeline.
"""

import yaml
import os
from typing import Dict, Any, List, Optional, Union
from pathlib import Path


class ConfigManager:
    """
    Configuration manager for handling YAML and JSON configuration files.
    
    Provides methods for:
    - Loading configuration from files
    - Validating configuration structure
    - Merging configurations
    - Environment variable substitution
    """
    
    def __init__(self, config_path: Optional[Union[str, Path]] = None):
        """
        Initialize configuration manager.
        
        Args:
            config_path: Path to configuration file
            config_dict: Configuration dictionary (alternative to file)
            
        Raises:
            ValueError: If neither config_path nor config_dict is provided
        """
        if config_path:
            self.config = self.load_config_file(config_path)
            self.config_path = str(config_path)
        else:
            raise ValueError("Either config_path or config_dict must be provided")
        
        # Substitute environment variables
        # self._substitute_environment_variables()

    @staticmethod
    def get_file_extension(file_path: Union[str, Path]) -> str:
        """
        Get file extension (without dot).
        
        Args:
            file_path: Path to file
            
        Returns:
            File extension in lowercase
        """
        return Path(file_path).suffix.lower().lstrip('.')
    
    @staticmethod
    def load_config_file(file_path: Union[str, Path]) -> Dict[str, Any]:
        """
        Load configuration from YAML or JSON file.
        
        Args:
            file_path: Path to configuration file
            
        Returns:
            Configuration dictionary
            
        Raises:
            FileNotFoundError: If file doesn't exist
            ValueError: If file format is not supported
        """
        extension = ConfigManager.get_file_extension(file_path)
        
        if extension in ['yaml', 'yml']:
            with open(file_path, 'r', encoding='utf-8') as f:
                return yaml.safe_load(f) or {}
        else:
            raise ValueError(f"Unsupported configuration file format: {extension}")
    
    def get(self, key: str, default: Any = None) -> Any:
        """
        Get configuration value with optional default.
        
        Args:
            key: Configuration key (supports dot notation for nested keys)
            default: Default value if key is not found
            
        Returns:
            Configuration value or default
        """
        return self._get_nested_value(self.config, key, default)
        
    @staticmethod
    def _get_nested_value(config: Dict[str, Any], key: str, default: Any = None) -> Any:
        """
        Get nested configuration value using dot notation.
        
        Args:
            config: Configuration dictionary
            key: Key with dot notation (e.g., 'task.params.value')
            default: Default value if key is not found
            
        Returns:
            Configuration value or default
        """
        keys = key.split('.')
        value = config
        
        for k in keys:
            if isinstance(value, dict) and k in value:
                value = value[k]
            else:
                return default
        
        return value
        
    def to_dict(self) -> Dict[str, Any]:
        """
        Get configuration as dictionary.
        
        Returns:
            Configuration dictionary
        """
        return self.config.copy()
    

    def get_pipeline_steps(config: dict, pipeline_name: str, document_format: str, step_name: str) -> dict:
        """
        Return the steps dictionary for a given pipeline and document format.
        """
        step_list = config.get(pipeline_name, {}).get(document_format, [])
        step_config = ConfigManager.get_step_config(step_list, step_name)
        return step_config


    @staticmethod
    def get_step_config(config: list, step_name: str) -> dict:
        """
        Fetch all variables for a given step from the workflow config.

        Args:
            config (list): List of step dictionaries.
            step_name (str): Step name to search for.

        Returns:
            dict: Config dictionary for the step, or {} if not found.
        """
        for step in config:
            if step.get("step") == step_name:
                return step
        return {}


## Get the task variables required for the workflow

In [ ]:

environment = dbutils.jobs.taskValues.get(
    taskKey="unified_workflow_setup", key="environment")
job_id = dbutils.jobs.taskValues.get(
    taskKey="unified_workflow_setup", key="databricks_job_id")
job_run_id = dbutils.jobs.taskValues.get(
    taskKey="unified_workflow_setup", key="databricks_job_run_id")

globals_config = json.loads(dbutils.jobs.taskValues.get(
    taskKey="unified_workflow_setup", key="globals_config"))
services_config = json.loads(dbutils.jobs.taskValues.get(
    taskKey="unified_workflow_setup", key="services_config"))
pipelines_config = json.loads(dbutils.jobs.taskValues.get(
    taskKey="unified_workflow_setup", key="pipelines_config"))
unified_steps_config = json.loads(dbutils.jobs.taskValues.get(
    taskKey="unified_workflow_setup", key="unified_steps_config"))
    
# get global configs
checkpoint_base_location = globals_config["checkpoint_base_location"]
checkpoint_location = f"{checkpoint_base_location}/document_conversion_unstructured_checkpoints"
catalog = globals_config["catalog"]
bronze_schema_name = globals_config["bronze_schema_name"]
silver_schema_name = globals_config["silver_schema_name"]
max_files_per_trigger = globals_config["default_batch_size"]

# get Unstructured config
unstructured_content_extraction_cfg = ConfigManager.get_pipeline_steps(
    pipelines_config, "unstructured", "pdf", "UnstructuredContentExtractionProcessor"
)
strategy = unstructured_content_extraction_cfg["strategy"]
infer_document_structure = unstructured_content_extraction_cfg["infer_document_structure"]
extract_image_block_types = unstructured_content_extraction_cfg["extract_image_block_types"]
extract_image_block_to_payload = unstructured_content_extraction_cfg["extract_image_block_to_payload"]
skip_infer_table_types = unstructured_content_extraction_cfg["skip_infer_table_types"]
infer_table_structure = unstructured_content_extraction_cfg["infer_table_structure"]

# get Common config
unified_content_extraction_cfg = unified_steps_config["conversion"]
output_table = unified_content_extraction_cfg["output_table"]
input_table = unified_content_extraction_cfg["submissions_table"]
submission_table = f"{catalog}.{bronze_schema_name}.{input_table}"
output_table = f"{catalog}.{silver_schema_name}.{output_table}"
output_submission_status = unified_content_extraction_cfg["submission_status"]
landing_zone_template = unified_content_extraction_cfg["landing_zone"]
micro_batch_limit = unified_content_extraction_cfg["micro_batch_limit"]
target_partitions = unified_content_extraction_cfg["target_partitions"]

result_schema = StructType([
    StructField("document_id",  StringType(),  True),
    StructField("submission_id",  StringType(),  True),
    StructField("status",  StringType(),  True),
    StructField("download_path", StringType(),  True),
    StructField("download_size_bytes",     LongType(),  True),
    StructField("download_timestamp",     TimestampType(),  True),
    StructField("created_at",     TimestampType(),  True),
    StructField("updated_at",     TimestampType(),  True),
    StructField("databricks_job_id",     StringType(),  True),
    StructField("databricks_job_run_id",     StringType(),  True),
    StructField("error_message",  StringType(),  True),
    StructField("document_format",  StringType(),  True)
])

In [ ]:
def execute_document_unstructured_conversion_step(submission_id, destination_landing_folder, file_path):

    try:
        elements = partition_pdf(
            filename=file_path,
            strategy=strategy,
            extract_image_block_types=extract_image_block_types,
            extract_image_block_to_payload=extract_image_block_to_payload,
            skip_infer_table_types=skip_infer_table_types,
            infer_table_structure=infer_table_structure,
            infer_document_structure=infer_document_structure,
        )
    except Exception as e:
        return {
            'submission_id': submission_id,
            'status': 'exception_in_generating_unstructred_elements',
            'action': "updated_status",
            'preferred_format': "json",
            'error_message': str(e)
        }

    file_name = file_path.split("/")[-1]
    file_name = file_name.replace("pdf", "json")
    storage_path = f"{destination_landing_folder}/{file_name}"
    Path(destination_landing_folder).mkdir(parents=True, exist_ok=True)

    try:
        elements_dicts = [el.to_dict() for el in elements]
        with open(storage_path, 'w', encoding='utf-8') as f:
            json.dump(elements_dicts, f, indent=4)  # indent for readability
    except Exception as e:
        return {
            'submission_id': submission_id,
            'status': 'exception_in_writing_unstructured_json',
            'action': "updated_status",
            'preferred_format': "json",
            'error_message': str(e)
        }        

    processed_record = {
        'status': 'content_extraction_completed',
        'document_id': str(uuid.uuid4()),
        'submission_id': submission_id,                        
        'download_path': storage_path,
        'download_size_bytes': None,
        'download_timestamp': datetime.now(),
        'created_at': datetime.now(),
        'updated_at': datetime.now(),
        'error_message': None,
        'preferred_format': 'json'
    }

    return processed_record


### foreach_batch


In [ ]:
def foreach_batch(batch_df, batch_id: int):
    if batch_df.count() == 0:
      print(f"→ Batch {batch_id}: empty")
      return
    
    print(f"executing {batch_df.count()} records in {batch_id}")

    # Access the Spark session from the DataFrame
    spark = batch_df.sparkSession

    # Deduplicate by latest per submission_id inside the microbatch
    w = Window.partitionBy("submission_id").orderBy(
        F.col("last_updated").desc_nulls_last())
    latest = (batch_df
              .withColumn("rn", F.row_number().over(w))
              .where("rn = 1")
              .drop("rn"))

    # Optional claim step to prevent parallel runs from double-processing:
    # Create a temp view and atomically mark a slice as 'claimed' by this job.
    latest.createOrReplaceTempView("micro_latest")

    limit_clause = f"LIMIT {int(micro_batch_limit)}" if micro_batch_limit else ""
    spark.sql(f"""
      MERGE INTO {submission_table} t
      USING (
        SELECT submission_id
        FROM micro_latest
        WHERE submission_status IN ('submitted', 'acodis_still_processing')
        {limit_clause}
      ) c
      ON t.submission_id = c.submission_id
      WHEN MATCHED THEN UPDATE SET
        t.submission_status = 'extraction_in_progress',
        t.last_updated      = current_timestamp()
    """)

    # Load only the rows claimed by this run id (ensures single consumer per row)
    claimed = (spark.read.table(submission_table)
               .where(F.col("submission_status") == "extraction_in_progress")
               .where(F.col("pipeline_id") == "unstructured"))

    if claimed.count() == 0:
        print(f"→ Batch {batch_id}: No rows claimed")
        return
  
    print(f"claimed {claimed.count()} records for extraction")

    claimed_df = claimed.collect()
    print(f"count of records being processed: {claimed.count()}")
    out_rows = []
    for row in claimed_df:
        submission_id = row.submission_id
        api_transaction_id = row.api_transaction_id
        pipeline_type = row.pipeline_id
        file_path = row.file_path
        # remove dbfs: prefix
        file_path = file_path[5:]
        destination_landing_zone = landing_zone_template.format(pipeline_id=pipeline_type, submission_id=submission_id)  

        try:             
            #raise RuntimeError("error in unstructured flow")
            result = execute_document_unstructured_conversion_step(submission_id, destination_landing_zone, file_path)                                        
            print(f"execute_document_conversion_step result::{submission_id}: {result}")

        except Exception as e:
            result = {
                'submission_id': submission_id,
                'status': 'failed - unstructured json extraction step',
                'action': "error_in_creating_unstructured_json",
                'error_message': f"Error extracting Unstructured Json: {str(e)}",
                'preferred_format': "json"
            }

        if result['status'] == 'content_extraction_completed':
            processed_record = {
                'document_id': str(uuid.uuid4()),
                'submission_id': submission_id,
                'status': "content_extraction_completed",
                'download_path': result['download_path'],
                'download_size_bytes': result['download_size_bytes'],
                'download_timestamp': datetime.now(),
                'created_at': datetime.now(),
                'updated_at': datetime.now(),
                'databricks_job_id': job_id,
                'databricks_job_run_id': job_run_id,
                'error_message': None,
                'document_format': result["preferred_format"]
            }
        else:
            processed_record = {
                'document_id': None,
                'submission_id': submission_id,
                'status': result['status'],
                'download_path': None,
                'download_size_bytes': None,
                'download_timestamp': None,
                'created_at': None,
                'updated_at': None,
                'databricks_job_id': job_id,
                'databricks_job_run_id': job_run_id,
                'error_message': result["error_message"],
                'document_format': result["preferred_format"]
            }

        out_rows.append(processed_record)


    print("creating result dataframe")
    result_df = spark.createDataFrame(out_rows, result_schema)
    print("created result dataframe")
       
    # for successfull downloads, append the results to output table
    result_df.write.format("delta").mode("append").saveAsTable(output_table)

    result_df.createOrReplaceTempView("submission_updates")
    # MERGE results back, clear claim, set result metadata
    spark.sql(f"""
      MERGE INTO {submission_table} AS t
      USING submission_updates u
      ON t.submission_id = u.submission_id
      WHEN MATCHED THEN UPDATE SET
        t.submission_status = u.status,
        t.last_updated = current_timestamp()
    """)



## Read Source Data
Read data from submissions table for submission_status=submitted

In [ ]:
# --- STREAM SOURCE ---
# Read inserts/updates via Change Data Feed so we re-check items moving through states for submission_status.
source_df =  (spark.readStream
             .format("delta")
             .option("readChangeFeed", "true")
             .table(submission_table)
             .filter(F.col("_change_type").isin("insert", "update_postimage"))
             .selectExpr("* EXCEPT(_change_type, _commit_version, _commit_timestamp)")
             )

# Only rows that need attention
work_df = source_df.where(F.col("submission_status").isin(["submitted"])).where(F.col("pipeline_id") == "unstructured")



## Write Data 
1. Write data to silver table
2. On success, update submission_status=content_extraction_completed

In [ ]:

streaming_query = (work_df.writeStream
                   .foreachBatch(foreach_batch)
                   .option("checkpointLocation", checkpoint_location)
                   .option("maxFilesPerTrigger", max_files_per_trigger) 
                   .trigger(availableNow=True)
                   .outputMode("append")
                   .start())

print("Document Conversion Job Completed started")

try:
    streaming_query.awaitTermination()
    print("Document Conversion Job Completed completed successfully")    
    
except Exception as e:
    print(f"Streaming job error: {str(e)}")
    streaming_query.stop()
    raise

